# Real Time Mask Detection (GITHUB)

In [1]:
import numpy as np
import imutils
from imutils.video import VideoStream
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import cv2
import dlib
import os
import time

## A function to detect the face and make the prediction

In [2]:
def detectnpredict(frame, net, model):
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300,300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    
    detected_faces = []
    locs = []
    predictions = []
    
    for i in range(len(detections[0,0])):
        confidence = detections[0,0,i,2]
        
        if confidence > conf_thresh:
            startx,starty,endx,endy = (detections[0,0,i,3:7] * np.array([w,h,w,h])).astype('int')
            detected_face = frame[starty:endy, startx:endx]
            detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2RGB)
        
            detected_face = cv2.resize(detected_face, (224,224)) 
       
            detected_face = img_to_array(detected_face)
        
            detected_face  = preprocess_input(detected_face)
            detected_faces.append(detected_face)
            locs.append((startx,starty,endx,endy))
            
    if len(detected_faces) > 0: 
        detected_faces = np.array(detected_faces, dtype = 'float32') #faces is a list. so converting into a numpy array
        predictions  = model.predict(detected_faces, batch_size = 32) 
    return locs,predictions
    
    

## Intialize and load the model

In [3]:
model = load_model('models/mob_model.h5')
caffe_path = 'models/res10_300x300_ssd_iter_140000.caffemodel'
proto_path = 'models/deploy.prototxt'
 
conf_thresh = 0.8

net = cv2.dnn.readNet(caffe_path, proto_path)

#start camera

vs = VideoStream(src = 0).start()
time.sleep(2.0)

In [ ]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width = 400)
    
    locs,preds = detectnpredict(frame, net, model)
    
    for box,pred in zip(locs,preds):
        startx,starty,endx, endy = box
        (mask,nomask) = pred
        
        label = "Mask" if mask > nomask else "No Mask"
        label  = "{}: {:.2f}%".format(label, max(mask,nomask)*100)
            
        color = (0,255,0) if mask> nomask else (0,0,255) #BGR form
        
            
        cv2.putText(frame, label, (startx,starty-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
        cv2.rectangle(frame, (startx,starty), (endx,endy), color, 2)
        
    
            
            
            
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

print("Recording has stopped")        
vs.stop()
cv2.destroyAllWindows()
cv2.waitKey(0)


# THE END